This is some quik-n-dirty code to get some rough numbers.
I'm not proud of it, but it does the job.

I neglect the shifts in the diagonals of the capactance matrix for
now, as they're all very minor.



## Boilerplate



Setup code and module imports



In [1]:
import numpy as np
import pint
from math import *
from cmc_formulas import *
import cmc_formulas.typical_values as T
from cmc_formulas.typical_values import *

## Typical Values



Here, we print the typical values that i've taken from
(Blais, Alexandre and Huang, Ren-Shou and Wallraff, Andreas and Girvin, S. M. and Schoelkopf, R. J., 2004) and (Gao, Yvonne Y. and Rol, M. Adriaan and Touzard, Steven and Wang, Chen, 2021).



### Resonator



In [1]:
T.CPW

namespace(Z_r=50 <Unit('ohm')>,
          v_0=130000000.0 <Unit('meter / second')>,
          d=1 <Unit('centimeter')>,
          l=153846.15384615381 <Unit('femtofarad / meter')>,
          c=384.6153846153846 <Unit('nanohenry / meter')>,
          C_r=1538.461538461538 <Unit('femtofarad')>,
          L_r=3.846153846153846 <Unit('nanohenry')>,
          ω_0=7.5398223686155035 <Unit('gigahertz')>)

In [1]:
T.CPW.ω_0 = Q_(1.2, "gigahertz") * 2 * np.pi

### Qubit



In [1]:
T.Transmon

namespace(E_C=200 <Unit('dirac_constant * megahertz * pi')>,
          E_J=10000 <Unit('dirac_constant * megahertz * pi')>,
          C_Σ=193.70229324659127 <Unit('femtofarad')>,
          ω_q=11.938052083641214 <Unit('gigahertz')>,
          α=0.6283185307179586 <Unit('gigahertz')>)

In [1]:
T.Transmon.ω_q / (2*np.pi)

1.9 gigahertz

In [1]:
(T.Transmon.E_J/T.Transmon.E_C).to(ureg.dimensionless)

50.0 dimensionless

In [1]:
formulas.ω_p(T.Transmon.E_C, T.Transmon.E_J) / (2*np.pi)

2.0 gigahertz

## Our Qubit



Let's make a standard transmon with a ratio of $E_J/E_C= 50$ and a
frequency of $\SI{5}{\giga\hertz}$.



In [1]:
transmon = T.TransmonQubit.from_angular_frequency_and_transmon_ratio(5 * ureg.gigahertz * 2 * np.pi, 50)
print(transmon.overview())

ω:      31.41592653589793 gigahertz
ω/2π:   5.0 gigahertz
E_J:    82.67349088394192 dirac_constant * gigahertz
E_C:    1.6534698176788385 dirac_constant * gigahertz
f_J:    13.157894736842106 gigahertz * planck_constant
f_C:    0.2631578947368421 gigahertz * planck_constant
ratio:  50.0 dimensionless

## Bus CPW



I'm just eyeballing this for now, but the bus should be below the
qubit so that we have some leeway with tuning the qubit interaction
and the readout strength.



In [1]:
bus = T.CoplanarWaveguide(length = 1.6 * ureg.cm)
bus.f

4.0625 gigahertz

This produces a detuning



In [1]:
T.Δ(transmon, bus)

5.890486225480863 gigahertz

The coherence time of Transmons in on the order of
$\SI{1}{\micro\second}$ (Blais, Alexandre and Grimsmo, Arne L. and Girvin, S. M. and Wallraff, Andreas, 2021), so that the oscillation
half-time $T_{1/2}$ time in [The Model for the CMC Project](../../knowledge/the_model_for_the_cmc_project.md) should be
shorter than that.



In [1]:
T_12 = 1 * ureg.microsecond

With this we can calculate the coupling capacitance between Transmon
and bus.



In [1]:
cc = formulas.C_c(T_12, transmon.C_Σ, transmon.E_J, bus.ω)
cc

1.124445239154514 femtofarad

The corresponding oscillator/qubit coupling strength $g$ is



In [1]:
T.g(transmon, bus, cc)

0.09619123726213973 gigahertz

It is important to keep $C_c\ll C_Σ,C_B$ and $g\ll \abs{Δ}\lesssim ω_q +
ω_B$. Otherwise, everything variale.
Also, the anharmonicity is much smaller than $Δ$. Basically
everything in the following should be small ($\sim .01$).



In [1]:
print(T.dispersive_metrics(transmon, bus, cc))

{'Anharmonicity, α/Δ': 0.28070175438596484, 'Coupling Strength, g/Δ': 0.016329931618554505, 'RWA counter-rotating': 0.10344827586206899, 'Coupling/Transmon Cap': 0.015276362345698466, 'Coupling/CPW Cap': 0.00045680587840652136}

As we can see, there is a lot of play in most things, but the
anharmonicity may be reduced. This can be done either by reducing the
qubit frequency while keeping the detuning constant or decreasing the
transmon ratio $E_J/E_C$. **Note that the CPW internal capacitance is
only a typical value here. I need more details on the actual design to
be more precise.**



## Drive



The readout's restriction is the anharmonicity, although this
constraint is weak, as we can use DRAG (Blais, Alexandre and Grimsmo, Arne L. and Girvin, S. M. and Wallraff, Andreas, 2021).

The question is: what is a typical pulse amplitude?
I've found numbers in (Bardin, Joseph, ????) that say $\SIrange{50}{500}{\micro\volt}$
are normal.

This is the (rough) limit on the coupling to the drive.



In [1]:
formulas.C_d_max(T.Transmon.E_C, T.Transmon.E_J, T.V_Dmax)

0.35825758655858 femtofarad

This is much smaller than the coupling to the bus and corresponds to
drive times that are much greater than



In [1]:
(2 * np.pi / T.Transmon.α).to(ureg.microsecond)

0.01 microsecond

which is still $1/10$ of $T_{1/2}$. This could get tight, but
thanks to DRAG this shouldn't be a big problem. Maybe we can also
detune the Qubit before flipping it.



## Readout



The readout should be **above** the qubit in frequency!!!!! This is *really*
important (see christobals talk). Basically

Let's start with the values from (Jeffrey, Evan and Sank, Daniel and Mutus, J. Y. and White, T. C. and Kelly, J. and Barends, R. and Chen, Y. and Chen, Z. and Chiaro, B. and Dunsworth, A. and Megrant, A. and O’Malley, P. J. J. and Neill, C. and Roushan, P. and Vainsencher, A. and Wenner, J. and Cleland, A. N. and Martinis, John M., 2014) (note that the
impedance of the cpw is $\SI{50}{\ohm}$) and i basically made up the
rest. The only things that enter are the frequency and the impedance.



In [1]:
readout = T.CoplanarWaveguide(length = .95 * ureg.cm)
readout.f

6.842105263157895 gigahertz

In [1]:
T.g(transmon, readout, cc) / (2*np.pi)

0.02038412287477586 gigahertz

In [1]:
print(T.dispersive_metrics(transmon, readout, cc))

{'Anharmonicity, α/Δ': 0.14285714285714285, 'Coupling Strength, g/Δ': 0.011065666703449752, 'RWA counter-rotating': 0.15555555555555556, 'Coupling/Transmon Cap': 0.012077024843609918, 'Coupling/CPW Cap': 0.0006082303260288571}